In [ ]:
#################################### Imports

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from IPython.display import display, IFrame, HTML
import os
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
import plotly.graph_objs as go
import pandas as pd
from dash.dependencies import Input, Output, State
import re
from nltk.util import ngrams
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import random
from collections import Counter
import plotly
import plotly.graph_objs as go
from plotly.offline import plot


#################################### HELPER FUNCTIONS

def freq(df, polarity, name):
    name_dict = {
    'JB':['justin', 'bieber'],
    'RH':['rihanna'],
    'KP':['katy', 'perry'],
    'TS':['taylorswift13', 'taylor', 'swift'],
#     'DT':['donald', 'trump']
    }
    dummy = " ".join(df[df['sentiment']==polarity]['tweet']).lower()
    for word in name_dict[name]:
        dummy = re.sub(word, '', dummy) 
    dummy = dummy.split(' ')
    dummy = [re.sub(r"[^a-zA-Z0-9]+", '', x) for x in dummy]
    dummy = [word for word in dummy if word not in stopwords.words('english')]
    dummy = [word for word in dummy if len(word)<=15]
    dummy = ' '.join(dummy)
    tokens = [token for token in dummy.split(" ") if token != ""]
    set_tokens = set(tokens)
    counts = Counter(tokens)
    most = counts.most_common(50)
    colors = [plotly.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(len(set_tokens))]
    total = sum(counts.values())
    weights = [x[1] for x in most] #[counts[i]*np.log(1000000*counts[i]) for i in set_tokens]
    tokie = [x[0] for x in most]
    return most, colors, weights, tokie

def convert(compound):
    if compound > 0.0:
        return 1
    elif compound < 0.0:
        return -1
    elif compound == 0.0:
        return 0
    
time = []
for h in range(24):
    for m in range(60):
        if h<10:
            H = '0'+str(h)
        else:
            H = str(h)
        if m<10:
            M = '0'+str(m)
        else:
            M = str(m)
        time.append(str(H)+':'+str(M))
        
def df_senti(df_name):
    df = pd.read_csv(df_name)
    df['date-time'] = df['date'] + ' ' + df['time']
    df['dt'] = pd.to_datetime(df['date-time'], errors='coerce')
    df = df[(df['dt']>datetime.date(2019,12,29)) & (df['dt']<datetime.date(2019,12,30))]
    df = df.sort_values(['dt']).reset_index()
    df['date_x'] = pd.to_datetime(df['dt']).dt.strftime('%H:%M')
    df['compound'] = [SentimentIntensityAnalyzer().polarity_scores(i)['compound'] for i in df['tweet']]
    df['sentiment'] = df['compound'].apply(lambda x: convert(x))
    grouped = df.groupby(df['date_x'])[['date_x','compound']].mean()
    df.to_csv('1day-senti-'+df_name)
    return df

JB = pd.read_csv('1day-senti-justinbieber-about.csv')
RH = pd.read_csv('1day-senti-rihanna-about.csv')
KP = pd.read_csv('1day-senti-katyperry-about.csv')
TS = pd.read_csv('1day-senti-taylorswift13-about.csv')
DT = pd.read_csv('senti-realDonaldTrump-about.csv')

data_dict = {
    'JB':JB,
    'RH':RH,
    'KP':KP,
    'TS':TS,
    'DT':DT,
}



def show_app(app, port=9999, width=900, height=1500):
    host = 'localhost'
    url = f'http://{host}:{port}'
    display(HTML(f"<a href='{url}' target='_blank'>Open in new tab</a>"))
    display(IFrame(url, width=width, height=height))
    app.css.config.serve_locally = True
    app.scripts.config.serve_locally = True
    return app.run_server(debug=False, host=host, port=port)

#################################### DASH APP

app = dash.Dash(assets_folder='static/', )
app.callback_map = {}


app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='About', children=[
            ###########
            html.Div([
                          html.H1('About Senti'),
                          html.Div('This tool will show and compare the sentiment of tweets that mentions celebrities using the vader sentiment tool. It will output how positive, neutral, and negative the sentiment of the input text is. The tool also shows a word cloud with the size of the word corresponding to the word\'s frequency.'),
                          html.Br(),
                          html.H1('What is Sentiment Analysis?'),
                          html.Div('Sentiment analysis (also known as opinion mining or emotion AI) refers to the use of natural language processing, text analysis, computational linguistics, and biometrics to systematically identify, extract, quantify, and study affective states and subjective information. Sentiment analysis is widely applied to voice of the customer materials such as reviews and survey responses, online and social media, and healthcare materials for applications that range from marketing to customer service to clinical medicine.'),
                          html.Div('Source: https://en.wikipedia.org/wiki/Sentiment_analysis'),
                          html.Br(),
                          html.H1('VADER-Sentiment-Analysis'),
                          html.Div('VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. It is fully open-sourced under the [MIT License] (we sincerely appreciate all attributions and readily accept most contributions, but please don\'t hold us liable).'),
                          html.Br(),
                          html.H2('About the Scoring'),
                          html.Div('The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a \'normalized, weighted composite score\' is accurate.'),
                          html.Br(),
                          html.Div('It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:'),
                          html.Li('positive sentiment: compound score >= 0.5'),
                          html.Li('neutral sentiment: (compound score > -0.5) and (compound score < 0.5)'),
                          html.Li('negative sentiment: compound score <= -0.5'),
                          html.Br(),   
                          html.Div('The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.'),
                          html.Div('Source: https://github.com/cjhutto/vaderSentiment'),
                          html.Br(),  
                          
                        ], style={'margin-right': '20%', 'margin-left': '20%'})
            ###########
        ]),
        dcc.Tab(label='Celebrity Twitter Sentiments', children=[
                        html.Br(),html.Br(),
                        
                        html.Div(dcc.Dropdown(id='dropdown-input',
                                                options=[
                                                    {'label': 'Justin Bieber', 'value': 'JB'},
                                                    {'label': 'Katy Perry', 'value': 'KP'},
                                                    {'label': 'Rihanna', 'value': 'RH'},
                                                    {'label': 'Taylor Swift', 'value': 'TS'},
#                                                     {'label': 'Donald Trump', 'value': 'DT'}
                                                ],
                                                value="JB"
                                            )),
                       
                        html.Div(#     Sentiment Line graph
                                  [html.Div([dcc.Graph(id='graph-time'),
                                             ]),
                                 #     Count Line graph
                                   html.Div([dcc.Graph(id='graph-line-count'),
                                             ])
                                  ], style={'width':'70%', 'display':'inline-block'}),
            
                        #     Bar graph
                        html.Div([html.Div([dcc.Graph(id='graph-counts'),
                                  ])], style={'width':'30%', 'display':'inline-block'}),
            
                        #     Word Cloud Negative
                        html.Div([dcc.Graph(id='cloud-neg'),
                                 ], style={'width':'50%', 'display':'inline-block'}),
                        #     Word Cloud Positive
                        html.Div([dcc.Graph(id='cloud-pos'),
                                 ], style={'width':'50%', 'display':'inline-block'}),
                        
            
                        #     Negative Words
                        html.Div([dcc.Graph(id='graph-neg'),
                                 ], style={'width':'50%', 'display':'inline-block'}),
                        #     Positive Words
                        html.Div([dcc.Graph(id='graph-pos'),
                                 ], style={'width':'50%', 'display':'inline-block'}),
            
                        
                        
        ]),
        
        dcc.Tab(label='Compare Celebrities', children=[
            
            ###########
            
            html.Div(id='tabs-content-classes'),
                        html.Div(dcc.Dropdown(id='dropdown-input-multi',
                                                options=[
                                                    {'label': 'Justin Bieber', 'value': 'JB'},
                                                    {'label': 'Katy Perry', 'value': 'KP'},
                                                    {'label': 'Rihanna', 'value': 'RH'},
                                                    {'label': 'Taylor Swift', 'value': 'TS'},
#                                                     {'label': 'Donald Trump', 'value': 'DT'}
                                                ],
                                                value=["JB"],
                                                multi=True,
                                            )  ),
                        #     Sentiment Line graph
                        html.Div([dcc.Graph(id='graph-time-multi'),
                                 ]),
                        #     Count Line graph
                        html.Div([dcc.Graph(id='graph-line-count-multi'),
                                 ]),
            
            ###########
            
        ]),
        
    ])
])

    
#################################### SINGLE CELEB


@app.callback(
    Output('graph-time', 'figure'),
    [Input('dropdown-input', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    grouped = df.groupby(df['date_x'])[['date_x','compound']].mean()
    data = [{'x': grouped.index.tolist(), 
             'y': grouped['compound'].tolist(),
            'type': 'line',
            'hovertemplate' :
            '<b>Time</b>: %{x}'+
            '<br><b>Average Compound<br>Sentiment Score</b>: %{y}<br>'}]

    return {'data': data, 
            'layout': {'title': 'Sentiment Score Trend',
                       'xaxis': {
                                   'title': 'Time',
                                },
                       'yaxis': {
                                    'title': 'Average Sentiment Score',
                                    'range': [-1,1]
                                },
                       'hovermode': 'closest',
                      }
           }

@app.callback(
    Output('graph-line-count', 'figure'),
    [Input('dropdown-input', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    grouped_count = df.groupby(df['date_x'])[['compound']].count()
    data = [{'x': grouped_count.index.tolist(), 
             'y': grouped_count['compound'].tolist(),
            'type': 'bar',
            'hovertemplate' :
            '<b>Time</b>: %{x}'+
            '<br><b>Tweet Counts</b>: %{y}<br>'}]

    return {'data': data, 
            'layout': {'xaxis': {
                                   'title': 'Time',
                                },
                       'yaxis': {
                                    'title': 'Tweet Counts'
                                },
                       'height':'250',
                       'hovermode': 'closest',
                      }
           }

@app.callback(
    Output('graph-counts', 'figure'),
    [Input('dropdown-input', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    y_val = df.groupby('sentiment')['tweet'].count().values.tolist()
    percent = np.round(df.groupby('sentiment')['tweet'].count().values/len(df)*100).tolist()
    percent = [str(np.round(i, 2))+'%' for i in percent]
    data = [{'x': ['Negative','Neutral','Positive'], 
             'y': y_val,
            'type': 'bar',
            'text': percent,
            'textposition':'auto',
            'hovertemplate' :
            '<b>Sentiment</b>: %{x}'+
            '<br><b>Counts</b>: %{y}'
             + '<br><b>Percent</b>: %{text}<br>'
            }]

    return {'data': data, 
            'layout': {'title': 'Sentiment Counts',
                       'xaxis': {
                                   'title': 'Sentiment',
                                },
                       'yaxis': {
                                   'title': 'Counts'
                                },
                       'height': '750',
                       'hovermode': 'closest',
                      }
           }

@app.callback(
    Output('cloud-neg', 'figure'),
    [Input('dropdown-input', 'value')],
)
def update_cloud(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    most, colors, weights, tokie = freq(df, -1, val_list)
    data = [{'x': random.sample(range(0, len(tokie)), len(tokie)), # [random.random() for i in range(len(set_tokens))],
             'y': random.sample(range(0, len(tokie)), len(tokie)), # [random.random() for i in range(len(set_tokens))],
             'mode':'text',
             'text':list(tokie),
             'textfont':{'size': weights,
                         'color': colors},
             'opacity': 0.8,
            'hovertemplate' :
            '<b>Word</b>: %{text}'+
            '<br><b>Frequency</b>: %{textfont.size:.0f}'}]
    
    return {'data': data, 
            'layout': {
                    'title': 'Negative Word Cloud',
                    'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                    'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}},
                    'hovermode': 'closest',
           }

@app.callback(
    Output('cloud-pos', 'figure'),
    [Input('dropdown-input', 'value')],
)
def update_cloud(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    most, colors, weights, tokie = freq(df, 1, val_list)
    data = [{'x': random.sample(range(0, len(tokie)), len(tokie)), # [random.random() for i in range(len(set_tokens))],
             'y': random.sample(range(0, len(tokie)), len(tokie)), # [random.random() for i in range(len(set_tokens))],
             'mode':'text',
             'text':list(tokie),
             'textfont':{'size': weights,
                         'color': colors},
             'opacity': 0.8,
            'hovertemplate' :
            '<b>Word</b>: %{text}'+
            '<br><b>Frequency</b>: %{textfont.size:.0f}'}]
    
    return {'data': data, 
            'layout': {
                    'title': 'Positive Word Cloud',
                    'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
                    'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}},
                    'hovermode': 'closest',
           }


@app.callback(
    Output('graph-neg', 'figure'),
    [Input('dropdown-input', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    most, colors, weights, tokie = freq(df, -1, val_list)
    data = [{'y': [x[0] for x in most],
             'x': [x[1] for x in most],
            'type': 'bar',
            'orientation':'h',
            'hovertemplate' :
            '<b>Word</b>: %{y}'+
            '<br><b>Frequency</b>: %{x}'}]

    return {'data': data, 
            'layout': {'title': 'Negative Words',
                       'yaxis': {
#                                    'title': 'Words',
                                   'autorange':"reversed",
                                   'automargin': True,
                                },
                       'xaxis': {
                                   'title': 'Frequency'
                                },
                       'hovermode': 'closest'
                      }
           }


@app.callback(
    Output('graph-pos', 'figure'),
    [Input('dropdown-input', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    df = data_dict[val_list]
    most, colors, weights, tokie = freq(df, 1, val_list)
    data = [{'y': [x[0] for x in most],
             'x': [x[1] for x in most],
            'type': 'bar',
            'orientation':'h',
            'hovertemplate' :
            '<b>Word</b>: %{y}'+
            '<br><b>Frequency</b>: %{x}'}]

    return {'data': data, 
            'layout': {'title': 'Positive Words',
                       'yaxis': {
                                   'autorange':"reversed",
                                   'automargin': True,
                                },
                       'xaxis': {
                                   'title': 'Frequency'
                                },
                       'hovermode': 'closest'
                      }
           }


#################################### MULTI CELEB
  
@app.callback(
    Output('graph-time-multi', 'figure'),
    [Input('dropdown-input-multi', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    
    df_day = pd.DataFrame(index=time)
    for each in ['JB','KP','TS','RH']:
        df = data_dict[each]
        grouped = df.groupby(df['date_x'])[['date_x','compound']].mean()
        df_day = df_day.merge(grouped[['compound']], how='outer', left_index=True, right_index=True)
    df_day.columns = ['JB','KP','TS','RH']
    data = []
    
    for choice in val_list:
        trace = {'x': df_day.index.tolist(), 
                 'y': df_day[choice].tolist(),
                'name':choice,
                'type': 'bar',
                'hovertemplate' :
                '<b>Celebrity</b>: '+ choice +
                '<br><b>Time</b>: %{x}'+
                '<br><b>Average Compound<br>Sentiment Score</b>: %{y}<br>',
                'opacity': 0.5}

        data.append(trace)

    return {'data': data, 
            'layout': {'title': 'Sentiment Score Trend',
                       'xaxis': {
                                  'title':'Time',
                                   'showticklabels': True,
                                },
                       'yaxis': {
                                    'title': 'Average Sentiment Score',
                                    'range': [-1,1]
                                },
                       'height':'600',
                       'hovermode': 'closest',
                      }
           }

@app.callback(
    Output('graph-line-count-multi', 'figure'),
    [Input('dropdown-input-multi', 'value')]
)
def update_graph(val_list):
    if not val_list:
        return {}
    
    df_day = pd.DataFrame(index=time)
    for each in ['JB','KP','TS','RH']:
        df = data_dict[each]
        grouped = df.groupby(df['date_x'])[['compound']].count()
        df_day = df_day.merge(grouped[['compound']], how='outer', left_index=True, right_index=True)
    df_day.columns = ['JB','KP','TS','RH']
    data = []
    
    for choice in val_list:
        trace = {'x': df_day.index.tolist(), 
                 'y': df_day[choice].tolist(),
                'type': 'bar',
                'hovertemplate' :
                '<b>Time</b>: %{x}'+
                '<br><b>Tweet Counts</b>: %{y}<br>',
                'opacity': 0.5,
                'name':choice}

        data.append(trace)

    return {'data': data, 
            'layout': {'xaxis': {
                                   'title': 'Time',
                                },
                       'yaxis': {
                                    'title': 'Tweet Counts'
                                },
                       'height':'300',
                       'hovermode': 'closest',
                      }
           }


show_app(app)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9999/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Dec/2019 08:39:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:43] "GET /_favicon.ico?v=1.6.0 HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2019 08:39:46] "POST /_dash-update-component HT

# Individual

In [125]:
# app.layout = html.Div([
#     html.Div(html.H1('SentiMo: Sentiment Monitoring Tool'), style={'text-align': 'center'}),
#     dcc.Tabs(
#         id="tabs-with-classes",
#         value='tab-3',
#         parent_className='custom-tabs',
#         className='custom-tabs-container',
#         children=[
#             dcc.Tab(
#                 label='About Sentiment Analysis',
#                 value='tab-3', className='custom-tab',
#                 selected_className='custom-tab--selected',
#                 children=[html.Div([
#                           html.H1('About SentiMo'),
#                           html.Div('This tool will accept a text input from the user and output the sentiment score using the vader sentiment tool. It will output how positive, neutral, and negative the sentiment of the input text is. It will also display the top 5 most positive and top 5 most negative n-grams. The tool also shows a word cloud with the size of the word corresponding to the word\'s frequency.'),
#                           html.Br(),
#                           html.H1('What is Sentiment Analysis?'),
#                           html.Div('Sentiment analysis (also known as opinion mining or emotion AI) refers to the use of natural language processing, text analysis, computational linguistics, and biometrics to systematically identify, extract, quantify, and study affective states and subjective information. Sentiment analysis is widely applied to voice of the customer materials such as reviews and survey responses, online and social media, and healthcare materials for applications that range from marketing to customer service to clinical medicine.'),
#                           html.Div('Source: https://en.wikipedia.org/wiki/Sentiment_analysis'),
#                           html.Br(),
#                           html.H1('VADER-Sentiment-Analysis'),
#                           html.Div('VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media. It is fully open-sourced under the [MIT License] (we sincerely appreciate all attributions and readily accept most contributions, but please don\'t hold us liable).'),
#                           html.Br(),
#                           html.H2('About the Scoring'),
#                           html.Div('The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a \'normalized, weighted composite score\' is accurate.'),
#                           html.Br(),
#                           html.Div('It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:'),
#                           html.Li('positive sentiment: compound score >= 0.5'),
#                           html.Li('neutral sentiment: (compound score > -0.5) and (compound score < 0.5)'),
#                           html.Li('negative sentiment: compound score <= -0.5'),
#                           html.Br(),   
#                           html.Div('The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.'),
#                           html.Div('Source: https://github.com/cjhutto/vaderSentiment'),
#                           html.Br(),  
#                           html.H1('What is an n-gram?'),
#                           html.Div('An n-gram is a contiguous sequence of n items from a given sample of text or speech. When the items are words, an n-gram of size 1 is referred to as a "unigram"; size 2 is a "bigram" (or, less commonly, a "digram"); size 3 is a "trigram". English cardinal numbers are sometimes used, e.g., "four-gram", "five-gram", and so on.'),
#                           html.Li('I (is a 1-gram)'),
#                           html.Li('I love (is a 2-gram)'),
#                           html.Li('I love to (is a 3-gram)'),
#                           html.Div('https://en.wikipedia.org/wiki/N-gram'),
#                         ], style={'margin-right': '20%', 'margin-left': '20%'})]
#             ),
            
#             dcc.Tab(
#                 label='SentiMo',
#                 value='tab-2',
#                 className='custom-tab',
#                 selected_className='custom-tab--selected',
#                 children=[
#                             html.Div(id='tabs-content-classes'),
#                         #     input
#                             html.Div(dcc.Textarea(
#                                                 placeholder='Enter your text...',
#                                                 id='text-input',
#                                                 value=
# """Yo
# His palms are sweaty, knees weak, arms are heavy
# There's vomit on his sweater already, mom's spaghetti
# He's nervous, but on the surface he looks calm and ready
# To drop bombs, but he keeps on forgettin'
# What he wrote down, the whole crowd goes so loud
# He opens his mouth, but the words won't come out
# He's chokin', how, everybody's jokin' now
# The clocks run out, times up, over, blaow!
# Snap back to reality, oh there goes gravity
# Oh, there goes Rabbit, he choked
# He's so mad, but he won't give up that easy? No
# He won't have it, he knows his whole back city's ropes
# It don't matter, he's dope, he knows that, but he's broke
# He's so stacked that he knows, when he goes back to his mobile home, that's when its
# Back to the lab again yo, this whole rhapsody
# He better go capture this moment and hope it don't pass him""",
#                                                 style={'width': '60%', 'height': '100px'}
#                                                 ), style={'text-align': 'center'}),
#                         #     bar graph
#                             html.Div([dcc.Graph(id='graph-ira', 
#                                                ),
#                                      ], style={'width':'40%', 'display':'inline-block'}),
#                         #     ngram compound score
#                             html.Div([dcc.Graph(id='graph-ira-3'),
#                                      ], style={'width':'60%', 'display':'inline-block'}),
#                             html.Div(' ', style={'width':'40%', 'display':'inline-block'}),
#                             html.Div([dcc.Slider(min=1,
#                                                  id='my-slider',
#                                                  max=5,
#                                                  step=1,
#                                                  value=2,
#                                                  marks={i: '{}'.format(i) for i in range(1,6,1)},
#                                                 ),
#                                       ], style={'width':'60%', 'display':'inline-block'}),
#                             html.Br(),html.Br(),html.Br(),
#                             html.Div([dcc.Graph(id='graph-cloud',
#                                                 figure={'layout': {'height':'500px'
#                                                                   }
#                                                         }
#                                                 )
#                                     ])
#                         ],              
#             ),
#         ]),
#     ])

# @app.callback(Output('tabs-content-classes', 'children'),
#               [Input('tabs-with-classes', 'value')])
# def render_content(tab):
#     if tab == 'tab-2':
#         return html.Div([
#             html.Br(),
#         ])
#     elif tab == 'tab-3':
#         return html.Div([
#             html.Br(),
#         ])

# @app.callback(
#     Output('graph-ira', 'figure'),
#     [Input('text-input', 'value')]
# )
# def update_graph(val_list):
#     if not val_list:
#         return {}
#     data = []
#     val_list = re.sub(r"\n", ' ', val_list)
#     val_list = val_list.lower().split(' ')
#     val_list = [re.sub(r"[^a-zA-Z0-9]+", '', x) for x in val_list]
#     val_list = [word for word in val_list if word not in stopwords.words('english')]
#     val_list = ' '.join(val_list)
#     score = (SentimentIntensityAnalyzer().polarity_scores(val_list))
#     data = [{'x': ['Negative','Neutral','Positive'], 
#              'y': [score['neg'],score['neu'],score['pos']],
#             'type': 'bar',
#             'name': 'val'}]
#     if score['compound'] >= 0.5:
#         result = 'The sentiment is positive!'
#     else:
#         result = 'The sentiment is negative!'
#     return {'data': data, 
#             'layout': {'title': result,
#                        'xaxis': {
#                                    'title': 'Sentiment',
#                                 },
#                        'yaxis': {
#                                    'title': 'Score',
#                                    'range': [0,1],
#                                 },
#                        'hovermode': 'closest',
#                       }
#            }

# @app.callback(
#     Output('graph-ira-3', 'figure'),
#     [Input('text-input', 'value'), dash.dependencies.Input('my-slider', 'value')],
# )

# def update_graph3(val_list,value):
#     if not val_list:
#         return {}
#     data = []
#     val_list = re.sub(r"\n", ' ', val_list)
#     val_list = val_list.lower().split(' ')
#     val_list = [re.sub(r"[^a-zA-Z0-9]+", '', x) for x in val_list]
#     val_list = [word for word in val_list if word not in stopwords.words('english')]
#     val_list = ' '.join(val_list)
#     tokens = [token for token in val_list.split(" ") if token != ""]
#     output = []
#     for gram in range(1,value+1):
#         output.extend(list(ngrams(tokens, gram)))
#     my_grams = [' '.join(x) for x in output]
#     my_grams_scores = []
#     for each in my_grams:
#         my_grams_scores.append(SentimentIntensityAnalyzer().polarity_scores(each)['compound'])
#     grams_zipped = list(zip(my_grams,my_grams_scores))
#     grams_zipped = sorted(grams_zipped, key=lambda tup: tup[1])
#     grams_zipped = grams_zipped[:10] + grams_zipped[-10:]
    
#     data = [{'y': [x[0] for x in grams_zipped], 
#              'x': [x[1] for x in grams_zipped],
#             'type': 'bar',
#             'name': 'val',
#             'orientation':'h'}]
    
#     return {'data': data, 
#             'layout': {'title': 'Compound Score of N-Grams',
#                        'xaxis': {
#                                    'title': 'Compound Score',
#                                    'range': [-1,1],
#                                    'automargin': True,
#                                 },
#                        'yaxis': {
#                                    'title': 'N-Grams',
#                                    'automargin': True,
#                                 },
#                        'hovermode': 'closest',
#                       }
#            }

# @app.callback(
#     Output('graph-cloud', 'figure'),
#     [Input('text-input', 'value')],
# )

# def update_cloud(val_list):
#     import plotly
#     import plotly.graph_objs as go
#     from plotly.offline import plot
#     import random
#     from collections import Counter
#     if not val_list:
#         return {}
#     data = []
#     val_list = re.sub(r"\n", ' ', val_list)
#     val_list = val_list.lower().split(' ')
#     val_list = [re.sub(r"[^a-zA-Z0-9]+", '', x) for x in val_list]
#     val_list = [word for word in val_list if word not in stopwords.words('english')]
#     val_list = ' '.join(val_list)
#     tokens = [token for token in val_list.split(" ") if token != ""]
#     set_tokens = set(tokens)
#     counts = dict(Counter(tokens))
#     colors = [plotly.colors.DEFAULT_PLOTLY_COLORS[random.randrange(1, 10)] for i in range(len(set_tokens))]
#     weights = [counts[i]*np.log(1000000*counts[i]) for i in set_tokens]
    
#     data = [{'x': random.sample(range(0, len(set_tokens)), len(set_tokens)), # [random.random() for i in range(len(set_tokens))],
#              'y': random.sample(range(0, len(set_tokens)), len(set_tokens)), # [random.random() for i in range(len(set_tokens))],
#                  'mode':'text',
#                  'text':list(set_tokens),
#                  'marker':{'opacity': 0.1},
#                  'textfont':{'size': weights,
#                            'color': colors, 'opacity': 0.1}}]
#     return {'data': data, 
#             'layout': {
#                     'title': 'Word Cloud Based on Frequency',
#                     'xaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False},
#                     'yaxis': {'showgrid': False, 'showticklabels': False, 'zeroline': False}},
#                     'hovermode': 'closest',
#            }


# # if __name__ == '__main__':
# #     app.run_server(debug=True)

# show_app(app)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://localhost:9999/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_0m1573524713.8.6.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_0m1573524713.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_table/bundle.v4_5_0m1573524716.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_0m1573524713.8.6.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_core_components/highlight.v1_5_0m1573524714.pack.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 03:46:08] "GET /_dash-component-suites/dash_html_components/dash_html_